In [1]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *
from src.data import *
from src.space import *
from src.mcmc import *

# https://dvr.capitol.texas.gov/
level    = 'cntyvtd'
proposal = 'planc2122'  # for 2010 enacted, use 'planc2100' for US congress, 'plans2100' for TX Senate, or 'planh2100' for TX House
contract = 10

# D = Data()
S = Space(level=level, proposal=proposal, contract=contract)
M = MCMC(gpickle=S.gpickle, nodes=S.tbls['nodes'],
         max_steps   = 3,
         random_seed = 0,
         defect_cap  = 1000000,
         pop_deviation_target=10)
# M.run_chain()
# M.post_process()

/home/jupyter/MathVGerrmandering_CMAT_2021
getting states
Get proposal    ... using existing table ... success!
Get nodes       ... processing ... using existing data table ... creating districts table ... 

KeyboardInterrupt: 

In [ ]:
try:
    import pandas_bokeh
except:
    os.system('pip install --upgrade pandas-bokeh')
    import pandas_bokeh
pandas_bokeh.output_notebook()

plan_cols = ['geoid', 'plan', 'district']
query = f"select {', '.join(plan_cols)} from {M.output} where plan < 2"
plans = run_query(query).pivot(index='geoid', columns='plan', values='district')

node_cols = ['geoid', 'polygon', 'county', 'total_pop', 'aland', 'density', 'polsby_popper']
query = f"select {', '.join(node_cols)} from {M.nodes}"
nodes = run_query(query).set_index('geoid')

In [ ]:
repl = {c:str(c) for c in plans.columns}
plans.rename(columns=repl, inplace=True)

geo = gpd.GeoSeries.from_wkt(nodes['polygon'], crs=crs_census).simplify(0.001).buffer(0) #<-- little white space @ .001 ~5.7 mb, minimal at .0001 ~10mb, with no white space ~37mb
gdf = nodes.drop(columns='polygon').join(plans, how='inner')
gdf = gpd.GeoDataFrame(gdf, geometry=geo).reset_index()
gdf.head(3)

In [ ]:
width = 600
xlim = [-106.2, -94.0]
ylim = [ 25.4 ,  36.6]
aspect = (ylim[1] - ylim[0]) / (xlim[1] - xlim[0])
height = round(aspect * width)

fig = gdf.plot_bokeh(
    colormap = "Category20",
    hovertool_string = '@geoid, @county<br>pop=@total_pop<br>density=@density{0.0}<br>land=@aland{0.0}<br>pp=@polsby_popper{0.0}',
    fill_alpha = 0.5,
    line_alpha = 0.05,
    return_html = True,
    show_figure = True,
    slider = list(repl.values()),
    slider_name = "PLAN",
    show_colorbar = False,
    xlim = xlim,
    ylim = ylim,
    figsize = (width, height),
)
# fig

In [ ]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *
from src.data import *
from src.space import *
from src.mcmc import *

# D = Data()
#             refresh_all=('crosswalks', 'assignments', 'shapes', 'census', 'elections', 'all')
#             refresh_all=('census', 'all')
#             refresh_tbl=('all'))
#             refresh_tbl=('crosswalks')
#            )

S = Space(level='cntyvtd', proposal='planc2100',
#               refresh_all=('graph'))
#               refresh_all=('nodes', 'graph'),)
             )
# # print(self.proposal, self.district_type)
M = MCMC(gpickle=S.gpickle, nodes=S.tbls['nodes'], max_steps=0, random_seed=0, defect_cap=1000000, pop_deviation_target=200)
M.run_chain()
M.post_process()
# for district_type, proposals in D.proposals_dict.items():
#     for proposal in proposals:
#         print(f'\n\n{district_type} {proposal}')
#         self = Space(level='cntyvtd',
# #                     district_type=district_type,
#                     proposal=proposal,
# #                     refresh_tbl = ('nodes', 'graphs')
#                    )
#         assert 1==2